# Proyecto 1: "Segmentación de Clientes según sus compras"

## Objetivo:
En este proyecto se va a realizar una Clasificación no supervisada a cerca de el tipo de clientes que compran en alguna tienda de retail via online. Con esto se ejecutará una estrategia para identificar al cliente y quizás hacer recomendaciones al respecto. Para lograr esto habrá que hacer una clasificación mediante KMeans y Gaussian Mixture Models para identificar los clientes, para luego hacer implementar un algoritmo de recomendación. 

## Pasos:
  1. Identificar las variables de dataset.
  2. procesar los datos creando nuevas variables y/o transformando variables.
  3. Implementar el modelo No-Supervisado.
  4. Analizar resultaods.
  5. Crear algoritmo de Recomendación.
  6. Analizar resultados.
  7. Probar en la Nube GCP.
  8. Subir a Github usando comandos git.

# 1.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [2]:
PATH = "/home/juanito/Documentos/Proyectos Hobby/Forecasting_and_GCP/online_retail_II.xlsx"
ds = pd.read_excel(PATH)

In [3]:
ds.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


**Variables:**

 -`Invoice`: Número de la factura, debería corresponder a un mismo  `ID Customer`.
 
 -`StockCode`: Corresponde al ID del producto.
 
 -`Description`: El producto en sí.
 
 -`Quantity`: Cantidad del producto (el stock).
 
 -`InvoiceDate`: Fecha y hora en la que se emitió la factura o boleta.
 
 -`Price`: Precio (Supongo que en Dólares).
 
 -`Customer ID`: ID del Cliente.
 
 -`Country`: Nacionalidad del cliente. 

**Exploración de datos:**

In [4]:
(ds.isna().sum()/ds.shape[0])*100 

Invoice         0.000000
StockCode       0.000000
Description     0.557225
Quantity        0.000000
InvoiceDate     0.000000
Price           0.000000
Customer ID    20.539488
Country         0.000000
dtype: float64

> Se procederá a quitar aquellos datos faltantes pertenecientes en su mayoría a los costumers (\~ 20\%), ya que pertenecen a nuestra variable dependiente o **target**.

In [5]:
ds.dropna(axis=0,inplace=True)
print((ds.isna().sum()/ds.shape[0])*100 )

Invoice        0.0
StockCode      0.0
Description    0.0
Quantity       0.0
InvoiceDate    0.0
Price          0.0
Customer ID    0.0
Country        0.0
dtype: float64


In [6]:
ds.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

In [7]:
ds['Description'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    3245
REGENCY CAKESTAND 3 TIER              1872
STRAWBERRY CERAMIC TRINKET BOX        1536
ASSORTED COLOUR BIRD ORNAMENT         1376
HOME BUILDING BLOCK WORD              1229
                                      ... 
BLUE OWL DECORATION                      1
PINK OWL DECORATION                      1
IVORY WALL CLOCK                         1
RED WALL CLOCK                           1
BAKING MOULD EASTER EGG MILK CHOC        1
Name: Description, Length: 4459, dtype: int64

In [8]:
ds['Customer ID'].value_counts()

14911.0    5710
17841.0    5114
14606.0    3927
14156.0    2710
12748.0    2665
           ... 
18174.0       1
14827.0       1
17645.0       1
16443.0       1
13222.0       1
Name: Customer ID, Length: 4383, dtype: int64

In [9]:
len(ds[ds['Country']=="United Kingdom"]["Customer ID"].unique())

4035

In [10]:
ds[ds['Customer ID']==13085.0]["Invoice"].unique()

array([489434, 489435, 490068, 490069, 496092, 496166, 'C527339'],
      dtype=object)

In [11]:
ds[ds['Customer ID']==14911.0]["Invoice"].unique()

array([489520, 490542, 490687, 490972, 'C490997', 'C491013', 'C491021',
       491160, 'C491619', 'C491827', 492015, 492650, 'C492651', 492749,
       492761, 'C492828', 'C493859', 'C493868', 493877, 493897, 'C493898',
       493900, 'C494356', 'C495054', 495055, 495056, 495073, 'C495231',
       495277, 495488, 495733, 495734, 495758, 496008, 496334, 496564,
       496584, 'C496586', 497337, 'C497338', 497339, 'C497930', 498021,
       'C498122', 498135, 498249, 498254, 498569, 498921, 'C498922',
       498935, 499067, 499919, 499922, 499921, 500123, 'C500698', 501043,
       501201, 501281, 'C501323', 501484, 501627, 501628, 501810,
       'C502292', 'C502332', 502334, 502422, 'C502452', 'C502454', 502455,
       502456, 502595, 502673, 503067, 'C504197', 'C504199', 504913,
       504915, 505522, 'C505800', 505806, 505884, 506290, 506308, 506730,
       506731, 'C506739', 506740, 'C506753', 507064, 507386, 507393,
       507396, 507478, 507673, 'C508911', 509154, 509536, 'C509629',
 

> Un solo cliente fue más de una vez a comprar a la empresa de retail en cuestión, debido a que posee diferentes facturas o boletas. Dicho lo anterior, habría que ajustar la base de datos de tal forma que en una fila esté representado las facturas que representan a un cliente y las  decripciones de cada factura. Además, habría que tener en cuenta la cantidad de productos o stock que compró el cliente.

> También cambiaré todo a strings ya que hay valores enteros y valores string dentro de esta variable.

In [15]:
ds = ds.astype({"Invoice":str})
ds[ds['Customer ID']==14911.0]["Invoice"].unique()

array(['489520', '490542', '490687', '490972', 'C490997', 'C491013',
       'C491021', '491160', 'C491619', 'C491827', '492015', '492650',
       'C492651', '492749', '492761', 'C492828', 'C493859', 'C493868',
       '493877', '493897', 'C493898', '493900', 'C494356', 'C495054',
       '495055', '495056', '495073', 'C495231', '495277', '495488',
       '495733', '495734', '495758', '496008', '496334', '496564',
       '496584', 'C496586', '497337', 'C497338', '497339', 'C497930',
       '498021', 'C498122', '498135', '498249', '498254', '498569',
       '498921', 'C498922', '498935', '499067', '499919', '499922',
       '499921', '500123', 'C500698', '501043', '501201', '501281',
       'C501323', '501484', '501627', '501628', '501810', 'C502292',
       'C502332', '502334', '502422', 'C502452', 'C502454', '502455',
       '502456', '502595', '502673', '503067', 'C504197', 'C504199',
       '504913', '504915', '505522', 'C505800', '505806', '505884',
       '506290', '506308', '506730'

In [16]:
ds[ds['Customer ID']==13085.0]['InvoiceDate'].head(60)

0       2009-12-01 07:45:00
1       2009-12-01 07:45:00
2       2009-12-01 07:45:00
3       2009-12-01 07:45:00
4       2009-12-01 07:45:00
5       2009-12-01 07:45:00
6       2009-12-01 07:45:00
7       2009-12-01 07:45:00
8       2009-12-01 07:46:00
9       2009-12-01 07:46:00
10      2009-12-01 07:46:00
11      2009-12-01 07:46:00
7739    2009-12-03 14:06:00
7740    2009-12-03 14:06:00
7741    2009-12-03 14:06:00
7742    2009-12-03 14:06:00
7743    2009-12-03 14:06:00
7744    2009-12-03 14:06:00
7745    2009-12-03 14:06:00
7746    2009-12-03 14:06:00
7747    2009-12-03 14:06:00
7748    2009-12-03 14:07:00
7749    2009-12-03 14:07:00
7750    2009-12-03 14:07:00
7751    2009-12-03 14:07:00
7752    2009-12-03 14:07:00
74247   2010-01-29 10:06:00
74248   2010-01-29 10:06:00
74249   2010-01-29 10:06:00
74250   2010-01-29 10:06:00
74251   2010-01-29 10:06:00
74252   2010-01-29 10:06:00
74253   2010-01-29 10:06:00
74254   2010-01-29 10:06:00
74255   2010-01-29 10:06:00
74256   2010-01-29 1

El formato de la columna fecha es _año-mes-día hora:min:seg_ Con esta información podríamos designar algún peso con respecto a la fecha en el cual se hicieron las compras; como por ejemplo sí hay productos que a tal fecha tuvieron diferente precio.

In [17]:
grupo = ds.groupby('StockCode')

In [18]:
grupo.get_group(85048)[['InvoiceDate','Price','Customer ID','Country']]

,InvoiceDate,Price,Customer ID,Country
0,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
400,2009-12-01 11:35:00,7.95,15461.0,United Kingdom
626,2009-12-01 11:50:00,7.95,16714.0,United Kingdom
1134,2009-12-01 12:35:00,6.95,17641.0,United Kingdom
1256,2009-12-01 12:55:00,6.95,17056.0,United Kingdom
...,...,...,...,...
522573,2010-12-09 08:34:00,7.95,14243.0,United Kingdom
523103,2010-12-09 12:32:00,7.95,14527.0,United Kingdom
523546,2010-12-09 14:05:00,7.95,17978.0,United Kingdom
524636,2010-12-09 15:23:00,7.95,16042.0,United Kingdom


In [19]:
grupo.get_group(85048)[['Country']].value_counts()

Country       
United Kingdom    260
EIRE               11
Spain               1
Switzerland         1
dtype: int64

In [20]:
grupo2 = ds.groupby(['StockCode','Country'])

In [21]:
grupo2.get_group((22041,'EIRE'))[['InvoiceDate','Price','Customer ID']]

,InvoiceDate,Price,Customer ID
109673,2010-03-03 12:21:00,2.55,14911.0
322110,2010-08-31 10:52:00,2.55,14911.0
392562,2010-10-14 17:51:00,2.55,14911.0


In [22]:
grupo2.get_group((85048,'Spain'))[['InvoiceDate','Price','Customer ID']]

,InvoiceDate,Price,Customer ID
404602,2010-10-20 14:28:00,7.95,12421.0


In [23]:
grupo2.get_group((85048,'Switzerland'))[['InvoiceDate','Price','Customer ID']]

,InvoiceDate,Price,Customer ID
447601,2010-11-10 13:19:00,7.95,12422.0


El precio exactamente varía según la fecha en el cual se compró el producto ¿Hay alguna relación con respecto al cliente y su nacionalidad?- Sí, Hay variación de precio por fecha y por Nacionalidad aunque no se aprecia una correlación muy relevante.


Siguiente paso será notar algunas correlaciones, ordenar, limpiar y preparar los datos para visualizar, y luego hacer un análisis respectivo. 

Como queremos que realizar una segmentación de clientes, entonces defininimos la columna `Customer ID` como nuestra `Clase`. 

Visualicemos un poco...

In [24]:
ds['Customer ID'].value_counts()

14911.0    5710
17841.0    5114
14606.0    3927
14156.0    2710
12748.0    2665
           ... 
18174.0       1
14827.0       1
17645.0       1
16443.0       1
13222.0       1
Name: Customer ID, Length: 4383, dtype: int64

Tengo un total de **4383 clientes**, con diferentes frecuencias. Como una técnica de validación de la segmentación de clientes podría usar como test aquellos clientes que solo han ido a comprar una sola vez.